In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

today = datetime.today().replace(hour=0, minute=0, second=0, microsecond=0).strftime("%Y%m%d")
today

'20250519'

## Import Connected TV TCs and add other device columns

### Group by main and sub categories to create `df_dev_component`

#### test cases will be joined by ", "

In [106]:
available_devices = ['Android Mobile', 'Apple Mobile', 'Android TV', 'Apple TV', 'Fire TV', 'Smart TV', 'Vizio TV', 'Roku', 'Web']

# Import Connect TV TCs and process accordingly
df_connectedTV_raw = pd.read_csv("data/KOCOWA_4.0_tc_connectedTV.csv").reset_index(drop=True)
df_connectedTV_raw = df_connectedTV_raw.rename(columns={"FireTV": "Fire TV", "ROKU": "Roku", "AndroidTV": "Android TV", "AppleTV": "Apple TV"})

# Import Web and Mobile TCs and process accordingly
df_webMobile_raw = pd.read_csv("data/KOCOWA_4.0_tc_webMobile_Edited.csv").reset_index(drop=True)
df_webMobile_raw = df_webMobile_raw.rename(columns={"WEB": "Web", "Apple_Mobile": "Apple Mobile", "Android_Phone": "Android Mobile"})
df_K4_webMobile = df_webMobile_raw.fillna(0).copy()

# Column adjustment: Connected TVs 
df_K4_connectedTV = df_connectedTV_raw.fillna(0).copy()
df_K4_connectedTV["Web"] = False
df_K4_connectedTV["Apple Mobile"] = False
df_K4_connectedTV["Android Mobile"] = False
df_K4_connectedTV["Smart TV"] = True
df_K4_connectedTV["Vizio TV"] = True
df_K4_connectedTV["project_name"] = "KOCOWA 4.0"

# Column adjustment: Web, Mobile
df_K4_webMobile["Android TV"] = False
df_K4_webMobile["Apple TV"] = False
df_K4_webMobile["Fire TV"] = False
df_K4_webMobile["Smart TV"] = False
df_K4_webMobile["Vizio TV"] = False
df_K4_webMobile["Roku"] = False
df_K4_webMobile["project_name"] = "KOCOWA 4.0"

df_K4_consolidated = pd.concat([df_K4_connectedTV, df_K4_webMobile], axis=0)
df_K4_consolidated["scope_of_dev"] = None

# Convert "UNTESTED" to True boolean flags
columns_to_check = df_K4_consolidated.columns[6:15]

# Replace "UNTESTED" with True and "비대상" with False in the specified columns
df_K4_consolidated[columns_to_check] = df_K4_consolidated[columns_to_check].replace({
    "UNTESTED": True,
    "비대상": False
})

df_K4_consolidated = df_K4_consolidated.rename(columns={
    "대분류": "main_category",
    "중분류": "sub_category",
    "소분류": "component",
    "Section": "scenario",
    "테스트 항목": "test_case",
    "기대결과": "expected_result"})

device_columns = ['Fire TV', 'Roku', 'Android TV', 'Apple TV', 'Web','Apple Mobile', 'Android Mobile', 'Smart TV', 'Vizio TV']

# If component = 0, then fill it with sub_category
for index, row in df_K4_consolidated.iterrows():
    main_category = row.main_category
    sub_category = row.sub_category
    component = row.component

    # if main_category == "Contents":
    #     main_category_rename = "Content Details"
    #     df_K4_consolidated.at[index, "main_category"] = main_category_rename
    # elif main_category == "Series Detailed Page":
    #     main_category_rename = "Content Details"
    #     df_K4_consolidated.at[index, "main_category"] = main_category_rename
    # elif main_category.strip() == "My KOCOWA Page":
    #     main_category_rename = "My KOCOWA"
    #     df_K4_consolidated.at[index, "main_category"] = main_category_rename

    # Fill missing sub categories and components
    if component == 0:
        if row.sub_category == 0:
            sub_category_rename = row.main_category
            component_rename = row.main_category
            df_K4_consolidated.at[index, "sub_category"] = sub_category_rename
            df_K4_consolidated.at[index, "component"] = component_rename
        else:
            component_rename = row.sub_category
            df_K4_consolidated.at[index, "component"] = component_rename

# Rule: If the name of the component is the same as main category, use "General Rule" tag
for index, row in df_K4_consolidated.iterrows():
    main_category = row.main_category
    sub_category = row.sub_category
    component = row.component

    if sub_category.strip().lower() == main_category.strip().lower():
        sub_category_rename = "General Rule"
        df_K4_consolidated.at[index, "sub_category"] = sub_category_rename
    elif sub_category.strip().lower() == component.strip().lower():
        component_rename = "General Rule"
        df_K4_consolidated.at[index, "component"] = component_rename
    
    '''
    # Rule for 4.0 Project: if main category is Content Details and My KOCOWA, True for all web and mobile platforms
    if main_category == "My KOCOWA Page":
        df_K4_connectedTV.at[index, "Web"] = True
        df_K4_connectedTV.at[index, "Apple Mobile"] = True
        df_K4_connectedTV.at[index, "Android Mobile"] = True
    elif main_category == "Content Details":
        df_K4_connectedTV.at[index, "Web"] = True
        df_K4_connectedTV.at[index, "Apple Mobile"] = True
        df_K4_connectedTV.at[index, "Android Mobile"] = True
    '''

for index, row in df_K4_consolidated.iterrows():
    main_cat = str(row.main_category).strip()
    sub_cat = str(row.sub_category).strip()
    component = str(row.component).strip()

    scope_of_dev = f"{sub_cat} > {component}"
    df_K4_consolidated.at[index, "scope_of_dev"] = scope_of_dev
    
# Grouping the dataframe by main_category, sub_category, and component
# Aggregating test_case by joining them with ", " and taking the maximum value for device_columns
df_dev_component = (
    df_K4_consolidated
    .groupby(['project_name', 'main_category', 'scope_of_dev'])
    .agg({
        'test_case': lambda x: '\n'.join(x),  # Combine test cases into a single string
        **{col: 'max' for col in device_columns}  # Take the maximum value for each device column
    })
    .reset_index()  # Reset the index to make it a regular dataframe
).sort_values(by = ['main_category','scope_of_dev']).copy()
        
col_orders = ['project_name','main_category', 'scope_of_dev', 'test_case', 'Fire TV', 'Roku', 'Android TV', 'Apple TV', 'Web','Apple Mobile', 'Android Mobile', 'Smart TV', 'Vizio TV']
df_dev_component_cleaned = df_dev_component[~df_dev_component.scope_of_dev.str.contains("General Rule", na=False)][col_orders].copy()

/var/folders/lk/3z63ntmd7jx9rxfvb1tyzjb00000gn/T/ipykernel_24619/1386700913.py:37: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_K4_consolidated[columns_to_check] = df_K4_consolidated[columns_to_check].replace({


In [107]:
len(list(df_dev_component_cleaned.scope_of_dev.unique()))

126

In [110]:
df_dev_component_cleaned

,project_name,main_category,scope_of_dev,test_case,Fire TV,Roku,Android TV,Apple TV,Web,Apple Mobile,Android Mobile,Smart TV,Vizio TV
0,KOCOWA 4.0,Contents,Episodes List Screen > Episode Detail,Comma 확인\nImage 형태의 Series Content Name 확인\nTe...,True,True,True,True,False,False,False,True,True
2,KOCOWA 4.0,Contents,Episodes List Screen > Thumbnail,Content Name 생략 확인,True,True,True,True,False,False,False,True,True
3,KOCOWA 4.0,Contents,Episodes Screen > Bottom Toast POP UP,Left Time 확인\nLeft Time stringset 확인\nTab Menu...,True,True,True,True,False,False,False,True,True
4,KOCOWA 4.0,Contents,Episodes Screen > Share POP UP,Episodes Tab Menu stringset 확인\nEpisodes Tab M...,True,True,True,True,False,False,False,True,True
5,KOCOWA 4.0,Contents,Episodes Screen > information POP UP,Variety Content Sorting Order 확인\nK-POP Conten...,True,True,True,True,False,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,KOCOWA 4.0,Terms of Use Page,Show Info Area > Keytalk Section,Back Button 확인\nBack Button stringset 확인\nBack...,True,True,True,True,False,False,False,True,True
336,KOCOWA 4.0,VARIETY Channel Page,Content List Area > Update Soon Screen,Speech Bubble icon 확인\nsub text 확인,True,True,True,True,False,False,False,True,True
340,KOCOWA 4.0,VARIETY Channel Page,Top Area > Sorting Button,Default Setting 확인\nSorting Button 확인\nSorting...,True,True,True,True,False,False,False,True,True
342,KOCOWA 4.0,WELCOME TO KOCOWA Page,Webcomic Area > Description Area,Home Button 선택 시 동작 확인\nHome Button 기능 확인\nSub...,True,True,True,True,False,False,False,True,True


### Initial export for checkup (csv and excel): processed_KOCOWA_4.0_tc_connectedTV

In [111]:
df_dev_component_cleaned.to_csv("data/processed/processed_KOCOWA_4.0_tc_connectedTV.csv", index=False)
# df_dev_component.to_excel("data/processed/processed_KOCOWA_4.0_tc_connectedTV.xlsx", index=False)